In [3]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import RobertaForSequenceClassification, AutoTokenizer
from tqdm import tqdm

# Configuration
MODEL_NAME = "roberta-base"
MAX_LEN = 256
BATCH_SIZE = 16  # Can increase for inference
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "model/toxic_roberta_best.pt"
THRESHOLD = 0.5

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Dataset class for inference (no labels)
class TestDataset:
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=MAX_LEN,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
        }

# Load model
print("Loading model...")
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=6)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# Load test data
print("Loading test data...")
test_df = pd.read_csv("data/test.csv")
sample_df = pd.read_csv("data/sample_submission.csv")

# Prepare test dataset
test_texts = test_df["comment_text"].fillna("none").values
test_dataset = TestDataset(test_texts)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Generate predictions
print("Generating predictions...")
all_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        probabilities = torch.sigmoid(outputs.logits).cpu().numpy()
        
        all_predictions.append(probabilities)

# Concatenate all predictions
predictions = np.concatenate(all_predictions, axis=0)

# Convert probabilities to binary predictions
binary_predictions = (predictions > THRESHOLD).astype(int)

# Create submission dataframe
submission_df = pd.DataFrame()
submission_df['id'] = test_df['id']

# Add binary prediction columns (using same column names as sample submission)
label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for i, col in enumerate(label_columns):
    submission_df[col] = binary_predictions[:, i]

# Save submission
submission_df.to_csv('data/submission.csv', index=False)
print("Submission saved to data/submission.csv")

# Display sample predictions
print("\nSample predictions:")
print(submission_df.head())
print(f"\nBinary prediction counts:")
for col in label_columns:
    positive_count = submission_df[col].sum()
    print(f"{col}: {positive_count}/{len(submission_df)} ({positive_count/len(submission_df)*100:.2f}% positive)")

Loading model...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\samak\AppData\Local\Temp\ipykernel_47824\1504788085.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode u

Loading test data...
Generating predictions...


Predicting: 100%|██████████| 9573/9573 [09:09<00:00, 17.41it/s]


Submission saved to data/submission.csv

Sample predictions:
                 id  toxic  severe_toxic  obscene  threat  insult  \
0  00001cee341fdb12      1             1        1       0       1   
1  0000247867823ef7      0             0        0       0       0   
2  00013b17ad220c46      0             0        0       0       0   
3  00017563c3f7919a      0             0        0       0       0   
4  00017695ad8997eb      0             0        0       0       0   

   identity_hate  
0              1  
1              0  
2              0  
3              0  
4              0  

Binary prediction counts:
toxic: 35976/153164 (23.49% positive)
severe_toxic: 6473/153164 (4.23% positive)
obscene: 24449/153164 (15.96% positive)
threat: 1052/153164 (0.69% positive)
insult: 22749/153164 (14.85% positive)
identity_hate: 5016/153164 (3.27% positive)
